# Requirements

- Python3.7
- Tesnorflow (`pip install tensorflow`)
- Tensorflow Dataset (`pip install tensorflow_datasets`)
- Requsets (`pip install requests`)
- Easytensor (`pip install easytensor`)
 ###### Note: this was adapted from https://www.tensorflow.org/datasets/keras_example

In [6]:
! pip install tensorflow tensorflow_datasets requests easytensor

     |████████████████████████████████| 173.9 MB 3.8 MB/s eta 0:00:0101   |████████████████████            | 108.6 MB 4.9 MB/s eta 0:00:14
     |████████████████████████████████| 3.7 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 4.8 MB/s eta 0:00:011
ERROR: Could not find a version that satisfies the requirement easytensor
ERROR: No matching distribution found for easytensor


In [7]:
!pwd

/Users/kamalkamalaldin/repos/python-client


In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /Users/kamalkamalaldin/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [2]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [3]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 3s 3ms/step - loss: 0.6138 - sparse_categorical_accuracy: 0.8252 - val_loss: 0.1922 - val_sparse_categorical_accuracy: 0.9441
Epoch 2/6
469/469 [==============================] - 1s 1ms/step - loss: 0.1808 - sparse_categorical_accuracy: 0.9493 - val_loss: 0.1386 - val_sparse_categorical_accuracy: 0.9602
Epoch 3/6
469/469 [==============================] - 1s 1ms/step - loss: 0.1176 - sparse_categorical_accuracy: 0.9662 - val_loss: 0.1109 - val_sparse_categorical_accuracy: 0.9662
Epoch 4/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0906 - sparse_categorical_accuracy: 0.9746 - val_loss: 0.0930 - val_sparse_categorical_accuracy: 0.9715
Epoch 5/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0721 - sparse_categorical_accuracy: 0.9802 - val_loss: 0.0872 - val_sparse_categorical_accuracy: 0.9752
Epoch 6/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0619 - sparse_categorical_accu

In [5]:
import os
export_path = os.path.join(os.getcwd(), "my_model")
print("export_path: {}".format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

export_path: /Users/kamalkamalaldin/repos/python-client/my_model
INFO:tensorflow:Assets written to: /Users/kamalkamalaldin/repos/python-client/my_model/assets


INFO:tensorflow:Assets written to: /Users/kamalkamalaldin/repos/python-client/my_model/assets


In [6]:
import easytensor
model_id, access_token = easytensor.upload_model("MNIST Model", export_path)

Access token is expired. Please reauthenticate.
Username: kamal@easytensor.com
Password: ········


In [7]:
print(model_id, access_token)

93f860ef-717d-4f85-8eae-12798c23710a df1dfb7f-bb35-498e-aca2-4bb46230f6fc


In [60]:
image_batches = list(map(lambda x: x[0], ds_train)) # shape (x, 128, 28, 28, 1)
label_batches = list(map(lambda x: x[1], ds_train)) # shape (y, 128, 28, 28, 1)
print("number of image batches:", len(image_batches))
print("Each batch contains", image_batches[0].shape[0], "images")
print("Each image has shape", image_batches[0][0].shape)

number of image batches: 469
Each batch contains 128 images
Each image has shape (28, 28, 1)


In [53]:
response = requests.post(
    "https://app.easytensor.com/query/",
    json={
        "instances": [
            # use the first picture of the first batch
            # make it a serializable list
            images[0][0].numpy().tolist()
        ]
    },
    headers={"accessToken": access_token}
)
print(response.text)

{"predictions":[[-2.39150405,-1.77103949,1.5249114,-5.33391428,2.85271907,-4.33721161,-0.218209475,-1.43464696,-4.75773859,-6.61607]]}
